In [2]:
import numpy as np
import torch.nn as nn
from PIL import Image
import glob
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import pandas as pd
import math

#img = Image.open("284.Bmp", "r")

N = 6283
D_in = 400
D_in2 = 20
H = 100
D_out = 62

x_list = []
x_data = np.zeros([N, D_in])
x_data2 = np.zeros([N, 1, D_in2, D_in2])
y_data = np.zeros([N, D_out])
y_data2 = np.zeros([N])

print("a")

for img_id in range(N + 1):
    for filename in glob.glob("C:\\Users\\ahmetcan\\Documents\\SWE\\SWE 582\\Google Street View\\trainResized\\" 
                              + str(img_id) + ".bmp"):
        im=Image.open(filename)
        pix = np.array(im)
        
        if len(pix.shape) > 2:
            pix2 = pix.mean(2)
            pix3 = pix2.reshape((1, D_in))
            x_data2[img_id - 1, 0, :, :] = pix2
        else:
            pix3 = pix.reshape((1, D_in))
            x_data2[img_id - 1, 0, :, :] = pix
            
        x_list.append(pix3)
        x_data[img_id - 1] = pix3

print("b")

a
b


In [3]:
def conv_lbl_to_int(a):
    if a.islower():
        return ord(a) - 61
    elif a.isnumeric():
        return int(a)
    else:
        return ord(a) - 55
    
df_labels = pd.read_csv('trainLabels.csv',header=0)

for index, row in df_labels.iterrows():
    if index < N:
        y_data2[index] = conv_lbl_to_int(row['Class'])

y_data2

x_tensor2 = torch.Tensor(x_data2)

y_tensor2 = torch.Tensor(y_data2)
y_tensor2 = y_tensor2.type(torch.LongTensor)

print("x")
y_tensor2

x



 49
  8
 29
 ⋮ 
 25
 23
 27
[torch.LongTensor of size 6283]

In [90]:
## Hyper Parameters
#num_epochs = 2
#batch_size = 100
#learning_rate = 0.001
#
# CNN Model (2 conv layer)
#class CNN(nn.Module):
#    def __init__(self):
#        super(CNN, self).__init__()
#        self.layer1 = nn.Sequential(
#            nn.Conv2d(1, 16, kernel_size=5, padding=2),
#            nn.BatchNorm2d(16),
#            nn.ReLU(),
#            nn.MaxPool2d(2))
#        self.layer2 = nn.Sequential(
#            nn.Conv2d(16, 32, kernel_size=5, padding=2),
#            nn.BatchNorm2d(32),
#            nn.ReLU(),
#            nn.MaxPool2d(2))
#        self.fc = nn.Linear(800, 62)
#        
#    def forward(self, x):
#        out = self.layer1(x)
#        out = self.layer2(out)
#        out = out.view(out.size(0), -1)
#        out = self.fc(out)
#        return out
#        
#cnn = CNN()
#
#
## Loss and Optimizer
#criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
#
## Train the Model
#for epoch in range(num_epochs):
#    #for i, (images, labels) in enumerate(train_loader):
#    images = Variable(x_tensor2)
#    labels = Variable(y_tensor2)
#    
#    # Forward + Backward + Optimize
#    optimizer.zero_grad()
#    outputs = cnn(images)
#    loss = criterion(outputs, labels)
#    loss.backward()
#    optimizer.step()
#    
#    print("loss: ", loss.data[0])
#    
#    #if (i+1) % 100 == 0:
#    #    print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
#    #           %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(80, 62)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = torch.optim.SGD(model.parameters(), lr=0.0005, momentum=0.99)

#rnd = np.random.randint(100, size=(N, 1, 3, 3))

#numberOfBatches = 2

batch_size = 600
numberOfBatches = math.ceil(N/batch_size)
idx_start = 0

#for batch_idx in range(numberOfBatches):
#    
#    #print("batch_idx: ", batch_idx)
#    #print("start: ", idx_start, "end: ", idx_start + batch_size)
#    #print("data: ", rnd[idx_start:idx_start + batch_size,:,:,:])
#    #print("shape: ", rnd[idx_start:idx_start + batch_size,:,:,:].shape)
#    idx_start = idx_start + batch_size

#print(x_data2[5:50,:,:,:].shape)
#print(y_data2.shape)
#print(y_data2[5:50].shape)

for epoch in range(1, 50):
    idx_start = 0
    for batch_idx in range(numberOfBatches):
        #train(epoch)
        #model.train()
        #for batch_idx, (data, target) in enumerate(train_loader):
        x_data3 = x_data2[idx_start:idx_start + batch_size,:,:,:]
        #print("batch_idx: ", batch_idx, "x-data3", x_data3)
        y_data3 = y_data2[idx_start:idx_start + batch_size]
        #print("batch_idx: ", batch_idx)
        #print("start: ", idx_start, "end: ", idx_start + batch_size)
        #print("x_data3.shape: ", x_data3.shape)
        #print("y_data3.shape: ", y_data3.shape)
        
        x_tensor3 = torch.Tensor(x_data3)
        #print("batch_idx: ", batch_idx, "x_tensor3", x_tensor3)
        
        y_tensor3 = torch.Tensor(y_data3)
        y_tensor3 = y_tensor3.type(torch.LongTensor)
        
        data, target = Variable(x_tensor3), Variable(y_tensor3)
        #print("batch_idx: ", batch_idx, "target", target)
        optimizer.zero_grad()
        output = model(data)
        #rint("batch_idx: ", batch_idx, "output", output)
        loss = F.nll_loss(output, target)
        print("batch_idx: ", batch_idx, "loss", loss)
        loss.backward()
        print("batch_idx: ", batch_idx, "loss_b", loss)
        optimizer.step()
        #print(output)
        #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #    epoch, batch_idx * len(data), batch_size,
        #    100. * batch_idx / batch_size, loss.data[0]))
        idx_start = idx_start + batch_size

batch_idx:  0 loss Variable containing:
 51.3042
[torch.FloatTensor of size 1]

batch_idx:  0 loss_b Variable containing:
 51.3042
[torch.FloatTensor of size 1]

batch_idx:  1 loss Variable containing:
 29.9425
[torch.FloatTensor of size 1]

batch_idx:  1 loss_b Variable containing:
 29.9425
[torch.FloatTensor of size 1]

batch_idx:  2 loss Variable containing:
 23.4876
[torch.FloatTensor of size 1]

batch_idx:  2 loss_b Variable containing:
 23.4876
[torch.FloatTensor of size 1]

batch_idx:  3 loss Variable containing:
 17.2716
[torch.FloatTensor of size 1]

batch_idx:  3 loss_b Variable containing:
 17.2716
[torch.FloatTensor of size 1]

batch_idx:  4 loss Variable containing:
 6.6430
[torch.FloatTensor of size 1]

batch_idx:  4 loss_b Variable containing:
 6.6430
[torch.FloatTensor of size 1]

batch_idx:  5 loss Variable containing:
 4.3941
[torch.FloatTensor of size 1]

batch_idx:  5 loss_b Variable containing:
 4.3941
[torch.FloatTensor of size 1]

batch_idx:  6 loss Variable cont

batch_idx:  8 loss_b Variable containing:
 4.0802
[torch.FloatTensor of size 1]

batch_idx:  9 loss Variable containing:
 4.0882
[torch.FloatTensor of size 1]

batch_idx:  9 loss_b Variable containing:
 4.0882
[torch.FloatTensor of size 1]

batch_idx:  10 loss Variable containing:
 4.0910
[torch.FloatTensor of size 1]

batch_idx:  10 loss_b Variable containing:
 4.0910
[torch.FloatTensor of size 1]

batch_idx:  0 loss Variable containing:
 4.0860
[torch.FloatTensor of size 1]

batch_idx:  0 loss_b Variable containing:
 4.0860
[torch.FloatTensor of size 1]

batch_idx:  1 loss Variable containing:
 4.0839
[torch.FloatTensor of size 1]

batch_idx:  1 loss_b Variable containing:
 4.0839
[torch.FloatTensor of size 1]

batch_idx:  2 loss Variable containing:
 4.0814
[torch.FloatTensor of size 1]

batch_idx:  2 loss_b Variable containing:
 4.0814
[torch.FloatTensor of size 1]

batch_idx:  3 loss Variable containing:
 4.0875
[torch.FloatTensor of size 1]

batch_idx:  3 loss_b Variable containi

batch_idx:  7 loss_b Variable containing:
 4.0174
[torch.FloatTensor of size 1]

batch_idx:  8 loss Variable containing:
 3.9907
[torch.FloatTensor of size 1]

batch_idx:  8 loss_b Variable containing:
 3.9907
[torch.FloatTensor of size 1]

batch_idx:  9 loss Variable containing:
 4.0067
[torch.FloatTensor of size 1]

batch_idx:  9 loss_b Variable containing:
 4.0067
[torch.FloatTensor of size 1]

batch_idx:  10 loss Variable containing:
 4.0195
[torch.FloatTensor of size 1]

batch_idx:  10 loss_b Variable containing:
 4.0195
[torch.FloatTensor of size 1]

batch_idx:  0 loss Variable containing:
 4.0186
[torch.FloatTensor of size 1]

batch_idx:  0 loss_b Variable containing:
 4.0186
[torch.FloatTensor of size 1]

batch_idx:  1 loss Variable containing:
 3.9984
[torch.FloatTensor of size 1]

batch_idx:  1 loss_b Variable containing:
 3.9984
[torch.FloatTensor of size 1]

batch_idx:  2 loss Variable containing:
 3.9940
[torch.FloatTensor of size 1]

batch_idx:  2 loss_b Variable containi

batch_idx:  4 loss Variable containing:
 3.9926
[torch.FloatTensor of size 1]

batch_idx:  4 loss_b Variable containing:
 3.9926
[torch.FloatTensor of size 1]

batch_idx:  5 loss Variable containing:
 3.9379
[torch.FloatTensor of size 1]

batch_idx:  5 loss_b Variable containing:
 3.9379
[torch.FloatTensor of size 1]

batch_idx:  6 loss Variable containing:
 3.9486
[torch.FloatTensor of size 1]

batch_idx:  6 loss_b Variable containing:
 3.9486
[torch.FloatTensor of size 1]

batch_idx:  7 loss Variable containing:
 3.9507
[torch.FloatTensor of size 1]

batch_idx:  7 loss_b Variable containing:
 3.9507
[torch.FloatTensor of size 1]

batch_idx:  8 loss Variable containing:
 3.9005
[torch.FloatTensor of size 1]

batch_idx:  8 loss_b Variable containing:
 3.9005
[torch.FloatTensor of size 1]

batch_idx:  9 loss Variable containing:
 3.9297
[torch.FloatTensor of size 1]

batch_idx:  9 loss_b Variable containing:
 3.9297
[torch.FloatTensor of size 1]

batch_idx:  10 loss Variable containing:

batch_idx:  0 loss_b Variable containing:
 3.9474
[torch.FloatTensor of size 1]

batch_idx:  1 loss Variable containing:
 3.8844
[torch.FloatTensor of size 1]

batch_idx:  1 loss_b Variable containing:
 3.8844
[torch.FloatTensor of size 1]

batch_idx:  2 loss Variable containing:
 3.8829
[torch.FloatTensor of size 1]

batch_idx:  2 loss_b Variable containing:
 3.8829
[torch.FloatTensor of size 1]

batch_idx:  3 loss Variable containing:
 3.9513
[torch.FloatTensor of size 1]

batch_idx:  3 loss_b Variable containing:
 3.9513
[torch.FloatTensor of size 1]

batch_idx:  4 loss Variable containing:
 3.9813
[torch.FloatTensor of size 1]

batch_idx:  4 loss_b Variable containing:
 3.9813
[torch.FloatTensor of size 1]

batch_idx:  5 loss Variable containing:
 3.9006
[torch.FloatTensor of size 1]

batch_idx:  5 loss_b Variable containing:
 3.9006
[torch.FloatTensor of size 1]

batch_idx:  6 loss Variable containing:
 3.9145
[torch.FloatTensor of size 1]

batch_idx:  6 loss_b Variable containing

...

batch_idx:  5 loss_b Variable containing:
 3.8672
[torch.FloatTensor of size 1]

batch_idx:  6 loss Variable containing:
 3.8749
[torch.FloatTensor of size 1]

batch_idx:  6 loss_b Variable containing:
 3.8749
[torch.FloatTensor of size 1]

batch_idx:  7 loss Variable containing:
 3.8821
[torch.FloatTensor of size 1]

batch_idx:  7 loss_b Variable containing:
 3.8821
[torch.FloatTensor of size 1]

batch_idx:  8 loss Variable containing:
 3.8491
[torch.FloatTensor of size 1]

batch_idx:  8 loss_b Variable containing:
 3.8491
[torch.FloatTensor of size 1]

batch_idx:  9 loss Variable containing:
 3.8597
[torch.FloatTensor of size 1]

batch_idx:  9 loss_b Variable containing:
 3.8597
[torch.FloatTensor of size 1]

batch_idx:  10 loss Variable containing:
 3.8667
[torch.FloatTensor of size 1]

batch_idx:  10 loss_b Variable containing:
 3.8667
[torch.FloatTensor of size 1]

batch_idx:  0 loss Variable containing:
 3.9097
[torch.FloatTensor of size 1]

batch_idx:  0 loss_b Variable containi

batch_idx:  3 loss_b Variable containing:
 3.9041
[torch.FloatTensor of size 1]

batch_idx:  4 loss Variable containing:
 3.9153
[torch.FloatTensor of size 1]

batch_idx:  4 loss_b Variable containing:
 3.9153
[torch.FloatTensor of size 1]

batch_idx:  5 loss Variable containing:
 3.8520
[torch.FloatTensor of size 1]

batch_idx:  5 loss_b Variable containing:
 3.8520
[torch.FloatTensor of size 1]

batch_idx:  6 loss Variable containing:
 3.8531
[torch.FloatTensor of size 1]

batch_idx:  6 loss_b Variable containing:
 3.8531
[torch.FloatTensor of size 1]

batch_idx:  7 loss Variable containing:
 3.8606
[torch.FloatTensor of size 1]

batch_idx:  7 loss_b Variable containing:
 3.8606
[torch.FloatTensor of size 1]

batch_idx:  8 loss Variable containing:
 3.8462
[torch.FloatTensor of size 1]

batch_idx:  8 loss_b Variable containing:
 3.8462
[torch.FloatTensor of size 1]

batch_idx:  9 loss Variable containing:
 3.8434
[torch.FloatTensor of size 1]

batch_idx:  9 loss_b Variable containing

batch_idx:  1 loss_b Variable containing:
 3.8312
[torch.FloatTensor of size 1]

batch_idx:  2 loss Variable containing:
 3.8516
[torch.FloatTensor of size 1]

batch_idx:  2 loss_b Variable containing:
 3.8516
[torch.FloatTensor of size 1]

batch_idx:  3 loss Variable containing:
 3.8878
[torch.FloatTensor of size 1]

batch_idx:  3 loss_b Variable containing:
 3.8878
[torch.FloatTensor of size 1]

batch_idx:  4 loss Variable containing:
 3.8949
[torch.FloatTensor of size 1]

batch_idx:  4 loss_b Variable containing:
 3.8949
[torch.FloatTensor of size 1]

batch_idx:  5 loss Variable containing:
 3.8451
[torch.FloatTensor of size 1]

batch_idx:  5 loss_b Variable containing:
 3.8451
[torch.FloatTensor of size 1]

batch_idx:  6 loss Variable containing:
 3.8398
[torch.FloatTensor of size 1]

batch_idx:  6 loss_b Variable containing:
 3.8398
[torch.FloatTensor of size 1]

batch_idx:  7 loss Variable containing:
 3.8469
[torch.FloatTensor of size 1]

batch_idx:  7 loss_b Variable containing

batch_idx:  0 loss_b Variable containing:
 3.8540
[torch.FloatTensor of size 1]

batch_idx:  1 loss Variable containing:
 3.8197
[torch.FloatTensor of size 1]

batch_idx:  1 loss_b Variable containing:
 3.8197
[torch.FloatTensor of size 1]

batch_idx:  2 loss Variable containing:
 3.8502
[torch.FloatTensor of size 1]

batch_idx:  2 loss_b Variable containing:
 3.8502
[torch.FloatTensor of size 1]

batch_idx:  3 loss Variable containing:
 3.8749
[torch.FloatTensor of size 1]

batch_idx:  3 loss_b Variable containing:
 3.8749
[torch.FloatTensor of size 1]

batch_idx:  4 loss Variable containing:
 3.8793
[torch.FloatTensor of size 1]

batch_idx:  4 loss_b Variable containing:
 3.8793
[torch.FloatTensor of size 1]

batch_idx:  5 loss Variable containing:
 3.8392
[torch.FloatTensor of size 1]

batch_idx:  5 loss_b Variable containing:
 3.8392
[torch.FloatTensor of size 1]

batch_idx:  6 loss Variable containing:
 3.8279
[torch.FloatTensor of size 1]

batch_idx:  6 loss_b Variable containing

batch_idx:  8 loss_b Variable containing:
 3.8315
[torch.FloatTensor of size 1]

batch_idx:  9 loss Variable containing:
 3.8129
[torch.FloatTensor of size 1]

batch_idx:  9 loss_b Variable containing:
 3.8129
[torch.FloatTensor of size 1]

batch_idx:  10 loss Variable containing:
 3.7970
[torch.FloatTensor of size 1]

batch_idx:  10 loss_b Variable containing:
 3.7970
[torch.FloatTensor of size 1]

batch_idx:  0 loss Variable containing:
 3.8389
[torch.FloatTensor of size 1]

batch_idx:  0 loss_b Variable containing:
 3.8389
[torch.FloatTensor of size 1]

batch_idx:  1 loss Variable containing:
 3.8059
[torch.FloatTensor of size 1]

batch_idx:  1 loss_b Variable containing:
 3.8059
[torch.FloatTensor of size 1]

batch_idx:  2 loss Variable containing:
 3.8469
[torch.FloatTensor of size 1]

batch_idx:  2 loss_b Variable containing:
 3.8469
[torch.FloatTensor of size 1]

batch_idx:  3 loss Variable containing:
 3.8633
[torch.FloatTensor of size 1]

batch_idx:  3 loss_b Variable containi

In [91]:
idxx = 181

print(target[idxx])

for i in range(283):
    print("------")
    out_numpy = output.data.numpy()
    print("argmax", np.argmax(out_numpy[i]))

Variable containing:
 27
[torch.LongTensor of size 1]

------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
argmax 10
------
arg

In [20]:
#print("x_data.shape", x_data.shape)

x_tensor = torch.Tensor(x_data)
#print("x_tensor", x_tensor)

x_data2[0, 0, :, :] = pix2

x_tensor2 = torch.Tensor(x_data2)
#print("x_tensor2", x_tensor2)

#print("pix2.shape: ", pix2.shape)

#print("x_data", x_data)
print("x_tensor", x_tensor)
#print("x_data2", x_data2)
print("x_tensor2", x_tensor2)

pix2

x_tensor 
  68.3333   69.3333   96.3333  ...    75.3333   75.6667   76.0000
 118.3333  118.6667  119.3333  ...   117.3333  117.3333  116.6667
  65.0000   79.0000   80.0000  ...    87.6667   77.6667   68.3333
             ...                ⋱                ...             
 121.0000  120.6667  120.6667  ...   118.3333  118.0000  119.3333
 196.3333  196.0000  196.0000  ...    60.3333   56.0000   81.3333
 113.6667  111.0000  111.3333  ...   109.3333  110.3333  108.6667
[torch.FloatTensor of size 6283x400]

x_tensor2 
( 0  , 0  ,.,.) = 
  113.6667  111.0000  111.3333  ...   110.3333  111.6667  113.0000
  111.0000  111.6667  114.3333  ...   111.0000  111.0000  110.6667
  108.3333  112.3333  113.0000  ...   113.0000  114.3333  113.3333
              ...                ⋱                ...             
  112.6667  111.3333  112.3333  ...   110.0000  109.6667  108.6667
  111.6667  111.0000  111.0000  ...   109.6667  110.0000  110.0000
  111.6667  113.6667  112.6667  ...   109.3333  110.3333  

array([[ 113.66666667,  111.        ,  111.33333333,  111.        ,
         109.66666667,  108.66666667,  114.33333333,  114.        ,
         113.66666667,  111.66666667,  111.66666667,  111.66666667,
         114.        ,  111.66666667,  108.66666667,  110.        ,
         109.66666667,  110.33333333,  111.66666667,  113.        ],
       [ 111.        ,  111.66666667,  114.33333333,  115.        ,
         115.33333333,  114.33333333,  111.66666667,  114.        ,
         113.        ,  114.33333333,  112.33333333,  111.        ,
         114.33333333,  113.        ,  111.        ,  112.66666667,
         114.66666667,  111.        ,  111.        ,  110.66666667],
       [ 108.33333333,  112.33333333,  113.        ,  111.66666667,
         109.66666667,  113.33333333,  110.33333333,  112.        ,
         111.66666667,  113.        ,  111.66666667,  112.66666667,
         113.33333333,  112.66666667,  112.33333333,  111.33333333,
         111.66666667,  113.        ,  114.333